In [2]:
import cv2
import pytesseract

# 设置Tesseract OCR识别引擎路径
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 读取图像
img = cv2.imread('car1.jpg')

# 转为灰度图
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 边缘检测
canny_img = cv2.Canny(gray_img, 100, 200)

# 寻找轮廓
contours, hierarchy = cv2.findContours(canny_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 筛选出符合条件的轮廓
for contour in contours:
    # 计算轮廓的周长
    perimeter = cv2.arcLength(contour, True)

    # 根据周长估算轮廓粗略的矩形框
    approximate_rect = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

    # 筛除不符合要求的轮廓
    if len(approximate_rect) == 4 and cv2.contourArea(contour) > 1000:
        x, y, w, h = cv2.boundingRect(contour)
        plate_img = img[y:y + h, x:x + w]

        # 在原始图像上绘制符合要求的矩形框
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # 计算车牌字符区域
        char_region = plate_img[int(h * 0.4):int(h * 0.8), int(w * 0.1):int(w * 0.9)]

        # 将车牌字符区域转为灰度图和二值化图
        gray_char_region = cv2.cvtColor(char_region, cv2.COLOR_BGR2GRAY)
        th_char_region = cv2.threshold(gray_char_region, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        # 调用Tesseract OCR识别车牌字符
        plate_number = pytesseract.image_to_string(th_char_region, config='--psm 10 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

        # 输出识别结果
        print("车牌号码为：" + plate_number)

cv2.imshow('result', img)
cv2.waitKey()
cv2.destroyAllWindows()


车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：CN

车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：T

车牌号码为：T

车牌号码为：P

车牌号码为：P

车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：OO

车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：
车牌号码为：I

车牌号码为：I

车牌号码为：
车牌号码为：
车牌号码为：14

车牌号码为：14

车牌号码为：
车牌号码为：
车牌号码为：


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [3]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model

model = load_model('mobilenet.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Pre-trained model for character recognition
# model = tf.keras.models.load_model('D:\code\python\machine_learning\cnn_zuo_tu.h5')

def segment_characters(thresholded_image):
    """Segment characters from the thresholded license plate image."""
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the original image to draw contours on it
    img_with_contours = np.copy(thresholded_image)

    # Define a list to store character images
    chars = []
    
    # Loop over all contours
    for contour in contours:
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Filter out contours with aspect ratio less than 0.2 or greater than 1.5
        aspect_ratio = w / h
        if (aspect_ratio < 0.2 or aspect_ratio > 1.5):
            continue
        
        # Extract the character image from the thresholded image
        char_image = thresholded_image[y:y+h, x:x+w]
        
        # Resize the character image to 28x28 pixels
        char_image = cv2.resize(char_image, (28, 28))
        
        # Add a border around the character image to match the input shape of the model
        char_image = cv2.copyMakeBorder(char_image, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        
        # Normalize the pixel values to be between 0 and 1
        char_image = char_image / 255.0
        
        # Add the normalized character image to the list of characters
        chars.append(char_image)
        
        # Draw a rectangle around the character on the original image
        cv2.rectangle(img_with_contours, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Show the original image with contours drawn on it
    cv2.imshow('Characters Segmented', img_with_contours)
    cv2.waitKey(0)
    
    return chars

def recognize(chars):
    """Recognize characters using a pre-trained model."""
    # Loop over all characters and use the pre-trained model to predict their label
    recognized_plate = ""
    for char in chars:
        # Reshape the character image to match the input shape of the model
        char = np.reshape(char, (1, 32, 32, 1))
        
        # Use the pre-trained model to predict the label of the character
        label = model.predict_classes(char)[0]
        
        # Convert the numerical label to the corresponding character
        recognized_plate += chr(label + 65)
    
    return recognized_plate


In [4]:
import cv2

# Load image
img = cv2.imread('car.jpg')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [5]:
# Apply Gaussian filter
blur = cv2.GaussianBlur(gray, (5, 5), 0)


In [6]:
# Apply Canny edge detection
edges = cv2.Canny(blur, 100, 200)


In [7]:
# Find contours in the edged image, keeping only the largest ones
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

# Loop over the contours
for contour in contours:
    # Approximate the contour
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)

    # If our approximated contour has four points, we can assume that we have found our license plate region
    if len(approx) == 4:
        plate_coords = approx
        break


In [8]:
# Extract the license plate from the image and apply thresholding to obtain a binary image
mask = np.zeros(gray.shape, dtype=np.uint8)
cv2.drawContours(mask, [plate_coords], -1, 255, -1)
mask = cv2.bitwise_and(gray, gray, mask=mask)
_, thresh = cv2.threshold(mask, 150, 255, cv2.THRESH_BINARY)

# Perform character segmentation and recognition
characters = segment_characters(thresh)
recognized_plate = recognize(characters)

print(recognized_plate)


AttributeError: 'Functional' object has no attribute 'predict_classes'

: 

In [1]:
from keras.applications import MobileNet

model = MobileNet(weights='imagenet')
model.save('mobilenet.h5')


17225924/17225924 [==============================] - 6s 0us/step
